In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from installFunctions import *
import pandas as pd
import numpy as np

In [3]:
from datetime import date

In [4]:
import holidays

In [5]:
gyra_fees = pd.read_csv('./install_csv/gyra_fees.csv')

In [6]:
import requests

In [7]:
import werkzeug

In [8]:
import datetime
rPaymentDate = datetime.datetime.strptime('08-08-2023', '%d-%m-%Y')
rDisbursementDate = datetime.datetime.strptime('09-05-2023', '%d-%m-%Y')
period = 12
amount = 17000
interest = 0.051

In [9]:
co = CreditOperation()

In [10]:
days_each_month, installDates, payDay = co.returnDaysEachMonth(period, rPaymentDate, rDisbursementDate)

In [11]:
days_each_month

[91, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30]

In [12]:
sum(days_each_month)

426

In [13]:
installDates,payDay

([datetime.date(2023, 8, 8),
  datetime.date(2023, 9, 8),
  datetime.date(2023, 10, 8),
  datetime.date(2023, 11, 8),
  datetime.date(2023, 12, 8),
  datetime.date(2024, 1, 8),
  datetime.date(2024, 2, 8),
  datetime.date(2024, 3, 8),
  datetime.date(2024, 4, 8),
  datetime.date(2024, 5, 8),
  datetime.date(2024, 6, 8),
  datetime.date(2024, 7, 8)],
 [datetime.date(2023, 8, 8),
  datetime.date(2023, 9, 8),
  datetime.date(2023, 10, 9),
  datetime.date(2023, 11, 8),
  datetime.date(2023, 12, 8),
  datetime.date(2024, 1, 8),
  datetime.date(2024, 2, 8),
  datetime.date(2024, 3, 8),
  datetime.date(2024, 4, 8),
  datetime.date(2024, 5, 8),
  datetime.date(2024, 6, 10),
  datetime.date(2024, 7, 8)])

In [14]:
amount = 17000
iof_fee_2 = amount*0.015
iof_fee_2 = iof_fee_2 + amount*0.0038
iof_fee_2

319.6

In [15]:
co.amortization_schedule(105000.0, 0.0199, period, rPaymentDate, rDisbursementDate)

([{'installDates': datetime.date(2023, 8, 8),
   'payDayDates': datetime.date(2023, 8, 8),
   'number': 1,
   'amortization_amount': 10346.822441,
   'interest': 6467.260438,
   'principal': 3879.562003,
   'balance': 97240.875994},
  {'installDates': datetime.date(2023, 9, 8),
   'payDayDates': datetime.date(2023, 9, 8),
   'number': 2,
   'amortization_amount': 10346.822441,
   'interest': 2080.058553,
   'principal': 8266.763888,
   'balance': 84586.91022},
  {'installDates': datetime.date(2023, 10, 8),
   'payDayDates': datetime.date(2023, 10, 9),
   'number': 3,
   'amortization_amount': 10346.822441,
   'interest': 1847.788115,
   'principal': 8499.034326,
   'balance': 75855.605456},
  {'installDates': datetime.date(2023, 11, 8),
   'payDayDates': datetime.date(2023, 11, 8),
   'number': 4,
   'amortization_amount': 10346.822441,
   'interest': 1735.184236,
   'principal': 8611.638205,
   'balance': 67131.363372},
  {'installDates': datetime.date(2023, 12, 8),
   'payDayDates': 

In [16]:
installments = co.amortization_schedule(amount, interest, period, rPaymentDate, rDisbursementDate)

In [42]:
co.find_pre_approved_with_fees(2000, 0.05, period, True, rPaymentDate, rDisbursementDate)

1
2
3
4
5
6
7


15742.073499548222

In [17]:
installments

([{'installDates': datetime.date(2023, 8, 8),
   'payDayDates': datetime.date(2023, 8, 8),
   'number': 1,
   'amortization_amount': 2143.273309,
   'interest': 2143.273309,
   'principal': 0.0,
   'balance': 17625.38341},
  {'installDates': datetime.date(2023, 9, 8),
   'payDayDates': datetime.date(2023, 9, 8),
   'number': 2,
   'amortization_amount': 2143.273309,
   'interest': 1555.017986,
   'principal': 588.255323,
   'balance': 15823.489355},
  {'installDates': datetime.date(2023, 10, 8),
   'payDayDates': datetime.date(2023, 10, 9),
   'number': 3,
   'amortization_amount': 2143.273309,
   'interest': 836.998979,
   'principal': 1306.27433,
   'balance': 13799.196016},
  {'installDates': datetime.date(2023, 11, 8),
   'payDayDates': datetime.date(2023, 11, 8),
   'number': 4,
   'amortization_amount': 2143.273309,
   'interest': 796.724088,
   'principal': 1346.549221,
   'balance': 12412.371905},
  {'installDates': datetime.date(2023, 12, 8),
   'payDayDates': datetime.date(20

In [18]:
preaprwfees = 17000
preapr = 17000
adjusted = True
iof_fee = True
interest_rate = 0.051
iofadjust = True

In [43]:
table, cet, acet, iofval, installAmount = co.calculate_amortization_schedule_with_taxes(preaprwfees, preapr, interest_rate, period, iofadjust, iof_fee, adjusted, rPaymentDate, rDisbursementDate)

In [45]:
co.calculate_amortization_schedule_with_taxes(15750.0,15000.0,0.0199,3,False,True,False, rPaymentDate, rDisbursementDate)

({'installDates': {0: datetime.date(2023, 8, 8),
   1: datetime.date(2023, 9, 8),
   2: datetime.date(2023, 10, 8)},
  'payDayDates': {0: datetime.date(2023, 8, 8),
   1: datetime.date(2023, 9, 8),
   2: datetime.date(2023, 10, 9)},
  'number': {0: 1, 1: 2, 2: 3},
  'amortization_amount': {0: 5740.122128, 1: 5740.122128, 2: 5740.122128},
  'interest': {0: 979.322554, 1: 229.132758, 2: 111.999638},
  'principal': {0: 4760.799574, 1: 5510.98937, 2: 5628.12249},
  'balance': {0: 11139.111861, 1: 5628.122492, 2: 0.0}},
 0.034726,
 0.506275,
 151.33832,
 5740.122128)

In [20]:
datetime.datetime(2023,5,9)-datetime.datetime(2024,4,9)

datetime.timedelta(days=-336)

In [21]:
installments = pd.json_normalize(
    [{'installDates': datetime.date(2023, 8, 8), 'payDayDates': datetime.date(2023, 8, 8), 'number': 1, 'amortization_amount': 2143.273309, 'interest': 2143.273309, 'principal': 0.0, 'balance': 17625.3834103699}, {'installDates': datetime.date(2023, 9, 8), 'payDayDates': datetime.date(2023, 9, 8), 'number': 2, 'amortization_amount': 2143.273309, 'interest': 1555.0179863657893, 'principal': 588.2553226342109, 'balance': 15823.489354731575}, {'installDates': datetime.date(2023, 10, 8), 'payDayDates': datetime.date(2023, 10, 9), 'number': 3, 'amortization_amount': 2143.273309, 'interest': 836.998978545654, 'principal': 1306.274330454346, 'balance': 13799.196016457096}, {'installDates': datetime.date(2023, 11, 8), 'payDayDates': datetime.date(2023, 11, 8), 'number': 4, 'amortization_amount': 2143.273309, 'interest': 796.7240878804099, 'principal': 1346.5492211195901, 'balance': 12412.37190467226}, {'installDates': datetime.date(2023, 12, 8), 'payDayDates': datetime.date(2023, 12, 8), 'number': 5, 'amortization_amount': 2143.273309, 'interest': 701.7049774153835, 'principal': 1441.5683315846168, 'balance': 10875.784462622618}, {'installDates': datetime.date(2024, 1, 8), 'payDayDates': datetime.date(2024, 1, 8), 'number': 6, 'amortization_amount': 2143.273309, 'interest': 649.6674015895516, 'principal': 1493.6059074104487, 'balance': 9330.140979386337}, {'installDates': datetime.date(2024, 2, 8), 'payDayDates': datetime.date(2024, 2, 8), 'number': 7, 'amortization_amount': 2143.273309, 'interest': 570.888536919661, 'principal': 1572.3847720803392, 'balance': 7678.977342636108}, {'installDates': datetime.date(2024, 3, 8), 'payDayDates': datetime.date(2024, 3, 8), 'number': 8, 'amortization_amount': 2143.273309, 'interest': 455.7111128691842, 'principal': 1687.562196130816, 'balance': 5876.2377224548145}, {'installDates': datetime.date(2024, 4, 8), 'payDayDates': datetime.date(2024, 4, 8), 'number': 9, 'amortization_amount': 2143.273309, 'interest': 398.9456437069649, 'principal': 1744.3276652930354, 'balance': 4075.14458799956}, {'installDates': datetime.date(2024, 5, 8), 'payDayDates': datetime.date(2024, 5, 8), 'number': 10, 'amortization_amount': 2143.273309, 'interest': 296.793084917922, 'principal': 1846.4802240820782, 'balance': 2126.511805128439}, {'installDates': datetime.date(2024, 6, 8), 'payDayDates': datetime.date(2024, 6, 10), 'number': 11, 'amortization_amount': 2143.273309, 'interest': 209.5517966625979, 'principal': 1933.7215123374021, 'balance': 105.54900453571281}, {'installDates': datetime.date(2024, 7, 8), 'payDayDates': datetime.date(2024, 7, 8), 'number': 12, 'amortization_amount': 2143.273309, 'interest': 104.00279636052873, 'principal': 2039.2705126394715, 'balance': 0.0}]
)

In [22]:
installments

,installDates,payDayDates,number,amortization_amount,interest,principal,balance
0,2023-08-08,2023-08-08,1,2143.273309,2143.273309,0.000000,17625.383410
1,2023-09-08,2023-09-08,2,2143.273309,1555.017986,588.255323,15823.489355
2,2023-10-08,2023-10-09,3,2143.273309,836.998979,1306.274330,13799.196016
3,2023-11-08,2023-11-08,4,2143.273309,796.724088,1346.549221,12412.371905
4,2023-12-08,2023-12-08,5,2143.273309,701.704977,1441.568332,10875.784463
5,2024-01-08,2024-01-08,6,2143.273309,649.667402,1493.605907,9330.140979
6,2024-02-08,2024-02-08,7,2143.273309,570.888537,1572.384772,7678.977343
7,2024-03-08,2024-03-08,8,2143.273309,455.711113,1687.562196,5876.237722
8,2024-04-08,2024-04-08,9,2143.273309,398.945644,1744.327665,4075.144588
9,2024-05-08,2024-05-08,10,2143.273309,296.793085,1846.480224,2126.511805


In [23]:
installments['payDayDates'].astype('datetime64').dt.strftime('%m/%d/%Y')

0     08/08/2023
1     09/08/2023
2     10/09/2023
3     11/08/2023
4     12/08/2023
5     01/08/2024
6     02/08/2024
7     03/08/2024
8     04/08/2024
9     05/08/2024
10    06/10/2024
11    07/08/2024
Name: payDayDates, dtype: object

In [24]:
iof_interest_rate=0.000041
iof_max_rate=0.015
iof_adicional=0.0038
accInterest = (installments['amortization_amount']-installments['interest']).apply(np.ceil)

days = [91,122,152,183,213,244,275,304,335,365,396,426]

iofs = []
for i in range(len(days)):
    if days[i] < 365:
        iofs.append(accInterest[i]*iof_interest_rate*days[i]+accInterest[i]*iof_adicional)
    else:
        iofs.append(accInterest[i]*iof_interest_rate*365+accInterest[i]*iof_adicional)
iofs,np.sum(iofs)

([0.0,
  5.184378000000001,
  13.111824,
  15.225141,
  18.072586,
  20.623176,
  23.712974999999997,
  27.453632000000002,
  30.598575,
  34.658955,
  36.29151,
  38.2806],
 263.213352)

In [25]:
co = CreditOperation()

In [26]:
days_each_month, installDates, payDay = co.returnDaysEachMonth(period, datetime.datetime.strptime('20-12-2020', '%d-%m-%Y'), datetime.datetime.strptime('20-11-2020', '%d-%m-%Y'))

In [27]:
co.calculate_amortization_schedule_with_taxes(16800, 16000, 0.0379, period=24, iof=False, rPaymentDate=rPaymentDate, rDisbursementDate=rDisbursementDate)

[91, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30]
[91, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30]


({'installDates': {0: datetime.date(2023, 8, 8),
   1: datetime.date(2023, 9, 8),
   2: datetime.date(2023, 10, 8),
   3: datetime.date(2023, 11, 8),
   4: datetime.date(2023, 12, 8),
   5: datetime.date(2024, 1, 8),
   6: datetime.date(2024, 2, 8),
   7: datetime.date(2024, 3, 8),
   8: datetime.date(2024, 4, 8),
   9: datetime.date(2024, 5, 8),
   10: datetime.date(2024, 6, 8),
   11: datetime.date(2024, 7, 8),
   12: datetime.date(2024, 8, 8),
   13: datetime.date(2024, 9, 8),
   14: datetime.date(2024, 10, 8),
   15: datetime.date(2024, 11, 8),
   16: datetime.date(2024, 12, 8),
   17: datetime.date(2025, 1, 8),
   18: datetime.date(2025, 2, 8),
   19: datetime.date(2025, 3, 8),
   20: datetime.date(2025, 4, 8),
   21: datetime.date(2025, 5, 8),
   22: datetime.date(2025, 6, 8),
   23: datetime.date(2025, 7, 8)},
  'payDayDates': {0: datetime.date(2023, 8, 8),
   1: datetime.date(2023, 9, 8),
   2: datetime.date(2023, 10, 9),
   3: datetime.date(2023, 11, 8),
   4: datetime.date(20

# IRR

In [271]:
initial_amount = 65000
array_irr = list(installments['amortization_amount'].values)
array_irr.insert(0,-initial_amount)

cet = np.irr(array_irr)
annualCet = (1+cet)**12-1

'cet ' + str(np.round(cet*100,4))+'%','Annual Cet ' + str(np.round(annualCet*100,4))+'%'

('cet 4.7253%', 'Annual Cet 74.0274%')

# XIRR

In [272]:
def xirr(transactions):
    years = [(ta[0] - transactions[0][0]).days / 365.0 for ta in transactions]
    residual = 1
    step = 0.05
    guess = 0.05
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, ta in enumerate(transactions):
            residual += ta[1] / pow(guess, years[i])
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return guess-1

In [273]:
array_irr = [ (i[1], i[0]) for i in installments[['amortization_amount','installDates']].values ]

In [274]:
array_irr.insert(0,(datetime.date(rDisbursementDate.year, rDisbursementDate.month, rDisbursementDate.day+1),-65000))

In [275]:
'Annual Cet ' + str(np.round(xirr(array_irr)*100,4))+'%'

'Annual Cet 68.2616%'

In [276]:
def xirr(df, guess=0.05, date_column='date', amount_column='amount'):
    '''Calculates XIRR from a series of cashflows. 
       Needs a dataframe with columns date and amount, customisable through parameters. 
       Requires Pandas, NumPy libraries'''

    df = df.sort_values(by=date_column).reset_index(drop=True)

    amounts = df[amount_column].values
    dates = df[date_column].values

    years = np.array(dates-dates[0], dtype='timedelta64[D]').astype(int)/365

    step = 0.05
    epsilon = 0.0001
    limit = 1000
    residual = 1

    #Test for direction of cashflows
    disc_val_1 = np.sum(amounts/((1+guess)**years))
    disc_val_2 = np.sum(amounts/((1.05+guess)**years))
    mul = 1 if disc_val_2 < disc_val_1 else -1

    #Calculate XIRR    
    for i in range(limit):
        prev_residual = residual
        residual = np.sum(amounts/((1+guess)**years))
        if abs(residual) > epsilon:
            if np.sign(residual) != np.sign(prev_residual):
                step /= 2
            guess = guess + step * np.sign(residual) * mul   
        else:
            return guess

In [277]:
'Annual Cet ' + str(np.round(xirr(pd.DataFrame(array_irr),0.04,0,1)*100,4))+'%'

'Annual Cet 68.2616%'

In [236]:
from scipy import optimize

In [237]:
def xnpv(rate, values, dates):
    '''Equivalent of Excel's XNPV function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xnpv(0.1, values, dates)
    -966.4345...
    '''
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]    # or min(dates)
    return sum([ vi / (1.0 + rate)**((di - d0).days / 365.0) for vi, di in zip(values, dates)])

def xirr(values, dates):
    '''Equivalent of Excel's XIRR function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xirr(values, dates)
    0.0100612...
    '''
    try:
        return scipy.optimize.newton(lambda r: xnpv(r, values, dates), 0.0)
    except RuntimeError:    # Failed to converge?
        return scipy.optimize.brentq(lambda r: xnpv(r, values, dates), -1.0, 1e10)

In [238]:
values = [ i[0] for i in installments[['amortization_amount','installDates']].values ]
dates = [ i[1] for i in installments[['amortization_amount','installDates']].values ]
values.insert(0,-65000)
dates.insert(0,datetime.date(rDisbursementDate.year, rDisbursementDate.month, rDisbursementDate.day))

In [239]:
xirr(values, dates)

0.6826159706073737

In [2]:
((0.740273849822884+1)**(1/12))-1

0.04725264567610643